# Task 3: Model Explainability with SHAP

This notebook demonstrates complete model explainability analysis using SHAP, including:
1. Built-in feature importance extraction
2. SHAP summary plot (global feature importance)
3. SHAP force plots for individual predictions (TP, FP, FN)
4. Comparison of SHAP vs built-in importance
5. Top 5 fraud prediction drivers
6. Business recommendations based on insights


In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Add src to path
sys.path.append(str(Path.cwd().parent / "src"))

from explainability_pipeline import ExplainabilityPipeline
from model_trainer import ModelTrainer
from model_evaluator import ModelEvaluator
from data_preparator import DataPreparator

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
%matplotlib inline

print("All modules imported successfully!")


## Step 1: Load Data and Model

Load preprocessed data and trained model


In [ ]:
# Load processed data
try:
    processed_df = pd.read_csv("../data/processed/processed_fraud_data.csv")
    print(f"Loaded processed data: {processed_df.shape}")
except FileNotFoundError:
    print("Processed data not found. Please run preprocessing pipeline first.")
    processed_df = None

# Load trained model (or train one if not available)
try:
    model_trainer = ModelTrainer()
    # Try to load best model
    best_model = model_trainer.load_model(
        "best_model",
        "../models/random_forest_best_model.joblib"
    )
    print("Loaded trained model")
except FileNotFoundError:
    print("Trained model not found. Training a model first...")
    if processed_df is not None:
        # Prepare data and train model
        data_prep = DataPreparator()
        target_col = "class" if "class" in processed_df.columns else "Class"
        X_train, X_test, y_train, y_test = data_prep.prepare_data(processed_df, target_col)
        
        model_trainer = ModelTrainer()
        best_model = model_trainer.train_random_forest(
            X_train, y_train,
            n_estimators=100,
            max_depth=10,
            class_weight='balanced',
            random_state=42
        )
        print("Model trained successfully")
    else:
        best_model = None


## Step 2: Initialize Explainability Pipeline


In [ ]:
# Initialize explainability pipeline
explainability_pipeline = ExplainabilityPipeline(
    output_dir="../models/explainability_outputs"
)

print("Explainability pipeline initialized")


## Step 3: Complete Explainability Analysis

Run the complete explainability pipeline


In [ ]:
if processed_df is not None and best_model is not None:
    # Prepare data
    data_prep = DataPreparator()
    target_col = "class" if "class" in processed_df.columns else "Class"
    X_train, X_test, y_train, y_test = data_prep.prepare_data(processed_df, target_col)
    
    # Run complete explainability analysis
    results = explainability_pipeline.explain_model(
        model=best_model,
        X_train=X_train,
        X_test=X_test,
        y_test=y_test,
        model_name="Random Forest",
        sample_size=1000
    )
    
    print("\nExplainability analysis complete!")
    print(f"Top 5 drivers identified: {len(results['top_drivers'])}")
    print(f"Force plots generated: {len(results['force_plots'])}")
    print(f"Recommendations: {len(results['recommendations'])}")
else:
    print("Cannot run explainability analysis. Please ensure data and model are available.")


## Step 4: Feature Importance Comparison

Compare built-in and SHAP feature importance


In [ ]:
if 'results' in locals() and results['comparison'] is not None:
    print("Feature Importance Comparison:")
    print("=" * 80)
    display(results['comparison'])
    
    print("\nKey Insights:")
    print(f"  - Built-in importance top feature: {results['builtin_importance'].iloc[0]['feature']}")
    print(f"  - SHAP importance top feature: {results['shap_importance'].iloc[0]['feature']}")
    
    # Check for differences
    builtin_top5 = set(results['builtin_importance'].head(5)['feature'])
    shap_top5 = set(results['shap_importance'].head(5)['feature'])
    common = builtin_top5 & shap_top5
    
    print(f"  - Common top 5 features: {len(common)}")
    if len(common) < 3:
        print("  - ⚠️ Significant difference between built-in and SHAP importance!")
    else:
        print("  - ✓ Good agreement between built-in and SHAP importance")


## Step 5: Top 5 Fraud Prediction Drivers

Analyze the top drivers of fraud predictions


In [ ]:
if 'results' in locals() and results['top_drivers']:
    print("Top 5 Fraud Prediction Drivers:")
    print("=" * 80)
    
    for i, driver in enumerate(results['top_drivers'], 1):
        print(f"\n{i}. {driver['feature']}")
        print(f"   Category: {driver['category']}")
        print(f"   SHAP Importance: {driver['importance']:.4f}")
        print(f"   Interpretation: {driver['interpretation']}")
    
    # Create summary dataframe
    drivers_df = pd.DataFrame(results['top_drivers'])
    display(drivers_df[['rank', 'feature', 'category', 'importance']])


## Step 6: Individual Prediction Analysis

Analyze specific cases: True Positive, False Positive, False Negative


In [ ]:
if 'results' in locals() and results['force_plots']:
    print("Individual Prediction Analysis:")
    print("=" * 80)
    
    for plot_info in results['force_plots']:
        print(f"\n{plot_info['prediction_type'].upper()} Case (Index: {plot_info['instance_idx']}):")
        print(f"  Predicted: {plot_info['predicted']}, Actual: {plot_info['actual']}")
        print(f"  Fraud Probability: {plot_info['probability']:.4f}")
        
        # Show top contributing features
        shap_vals = plot_info['shap_values']
        feature_vals = plot_info['feature_values']
        
        # Get top 5 contributing features
        feature_names = list(feature_vals.keys())
        top_contrib_idx = np.argsort(np.abs(shap_vals))[-5:][::-1]
        
        print(f"\n  Top 5 Contributing Features:")
        for idx in top_contrib_idx:
            feature = feature_names[idx]
            shap_val = shap_vals[idx]
            feat_val = feature_vals[feature]
            direction = "increases" if shap_val > 0 else "decreases"
            print(f"    - {feature}: {shap_val:.4f} ({direction} fraud probability)")
            print(f"      Feature value: {feat_val}")
    
    print("\n✓ Force plots saved to models/explainability_outputs/")
    print("  View the PNG files for detailed visualizations")


## Step 7: Business Recommendations

Review actionable business recommendations based on SHAP insights


In [ ]:
if 'results' in locals() and results['recommendations']:
    print("Business Recommendations Based on SHAP Analysis:")
    print("=" * 80)
    
    for i, rec in enumerate(results['recommendations'], 1):
        print(f"\nRecommendation {i}: {rec['title']}")
        print("-" * 80)
        print(f"{rec['recommendation']}")
        print(f"\nAction: {rec['action']}")
        print(f"Expected Impact: {rec['expected_impact']}")
        print(f"SHAP Insight: {rec['shap_insight']}")
        print()
    
    print("\n✓ Full recommendations saved to models/explainability_outputs/business_recommendations.txt")


## Summary

This analysis provides:
- ✅ Built-in feature importance visualization
- ✅ SHAP summary plot (global feature importance)
- ✅ SHAP force plots for TP, FP, FN cases
- ✅ Comparison of built-in vs SHAP importance
- ✅ Top 5 fraud prediction drivers identified
- ✅ Actionable business recommendations with SHAP justification

All outputs are saved to `models/explainability_outputs/` directory.
